<a href="https://colab.research.google.com/github/asungii/quantization-experiments/blob/main/SmoothQuant_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Installations</h1>

From SmoothQuant repo. Because the specified CUDA compiler is 11.3, and the default Colab CUDA version is as of 7/29/24 12.2, we have to change the CUDA version in Colab.

I found info on how to do this from:

https://medium.com/@ajithkumarv/how-to-modify-cuda-gcc-python-versions-in-colab-584ed4113157

https://github.com/googlecolab/colabtools/issues/4214#issuecomment-1859155789

https://vitalitylearning.medium.com/running-cuda-in-google-colab-525a92efcf75

In [1]:
# setup is from the mit-han-lab GitHub

# these bits are not necessary for Google Colab
# !conda create -n smoothquant python=3.8
# !conda activate smoothquant
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install transformers==4.36.0 accelerate datasets zstandard

# !python setup.py install

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled torch-2.3.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.1+cu121
    Uninstalling torchvision-0.18.1+cu121:
      Successfully uninstalled torchvision-0.18.1+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.1+cu121
    Uninstalling torchaudio-2.3.1+cu121:
      Successfully uninstalled torchaudio-2.3.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [26]:
import torch

In [2]:
! nvcc --version

# well it's 12.2. that's a bit of an issue

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.3.0/local_installers/cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu2004-11-3-local/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda-11.3

!nvcc --version

--2024-07-29 14:29:49--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2004.pin’

cuda-ubuntu2004.pin 100%[===================>]     190  --.-KB/s    in 0s      

2024-07-29 14:29:49 (5.84 MB/s) - ‘cuda-ubuntu2004.pin’ saved [190/190]

--2024-07-29 14:29:49--  https://developer.download.nvidia.com/compute/cuda/11.3.0/local_installers/cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O

In [23]:
#!export CUDA_PATH=/usr/local/cuda-11.3/
#!export PATH=/usr/local/cuda-11.3/bin:$PATH
#!export CUDA_PATH=/usr/local/cuda-11.3/bin:$PATH
#!export CUDA_HOME=/usr/local/cuda-11.3/bin:$PATH
#!export CUDA_HOME=/usr/local/cuda-11.3/

Claude says that the !export doesn't persist outside of the cell. Whoops.

In [27]:
print(torch.version.cuda)

11.3


In [28]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.3'
os.environ['PATH'] = '/usr/local/cuda-11.3/bin:' + os.environ['PATH']

# THIS WORKED! YAY!!

Now, we install a GCC version compatible:

In [48]:
!sudo apt-get update
!sudo apt-get install build-essential software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt-get update
!sudo apt-get install gcc-9 g++-9 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 90 --slave /usr/bin/g++ g++ /usr/bin/g++-9
!gcc -v

Get:1 file:/var/cuda-repo-ubuntu2004-11-3-local  InRelease
Ign:1 file:/var/cuda-repo-ubuntu2004-11-3-local  InRelease
Get:2 file:/var/cuda-repo-ubuntu2004-11-3-local  Release [564 B]
Get:2 file:/var/cuda-repo-ubuntu2004-11-3-local  Release [564 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:14 https://ppa.launchpadcontent.net/ubuntu-toolchain-r

This works, but you will have to deal with this girthy installation every time you want to use this notebook.

In [29]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Mar_21_19:15:46_PDT_2021
Cuda compilation tools, release 11.3, V11.3.58
Build cuda_11.3.r11.3/compiler.29745058_0


In [5]:
%cd /usr/local/

/usr/local


In [6]:
%pwd

'/usr/local'

In [7]:
%ls

bin/    cuda@     cuda-11.3/  cuda-12.2/  games/               include/  lib64/  opt/   share/
colab/  cuda-11@  cuda-12@    etc/        _gcs_config_ops.so/  lib/      man@    sbin/  src/


In [10]:
! echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [15]:
! python -c "import torch; print(torch.__version__)"

1.12.1+cu113


In [33]:
%cd /content

/content


In [34]:
! git clone https://github.com/mit-han-lab/smoothquant.git

Cloning into 'smoothquant'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 352 (delta 130), reused 117 (delta 101), pack-reused 164
Receiving objects: 100% (352/352), 6.80 MiB | 25.98 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [35]:
%cd smoothquant

/content/smoothquant


In [36]:
!pip install -e .

Obtaining file:///content/smoothquant
  Preparing metadata (setup.py) ... done
  Attempting uninstall: smoothquant
    Found existing installation: smoothquant 0.0.0
    Uninstalling smoothquant-0.0.0:
      Successfully uninstalled smoothquant-0.0.0
  Running setup.py develop for smoothquant


In [37]:
%cd ..

/content


In [38]:
! git clone https://github.com/Guangxuan-Xiao/torch-int.git

Cloning into 'torch-int'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 1102 (delta 139), reused 118 (delta 118), pack-reused 929
Receiving objects: 100% (1102/1102), 960.91 KiB | 5.89 MiB/s, done.
Resolving deltas: 100% (643/643), done.


In [39]:
%cd torch-int

/content/torch-int


I also had to jump through some hoops trying to figure out this submodule stuff—it is not exactly what the repo says to do

In [40]:
! git config submodule.submodules/cutlass.url https://github.com/NVIDIA/cutlass.git
! git submodule update --init --recursive

Cloning into '/content/torch-int/submodules/cutlass'...
Submodule path 'submodules/cutlass': checked out 'c975e2ccbb2dbf13024568b37ffa3498ed0b3aed'


In [44]:
!sudo update-alternatives --display cuda

!sudo update-alternatives --config cuda

cuda - auto mode
  link best version is /usr/local/cuda-12.2
  link currently points to /usr/local/cuda-12.2
  link cuda is /usr/local/cuda
/usr/local/cuda-11.3 - priority 113
/usr/local/cuda-12.2 - priority 122
There are 2 choices for the alternative cuda (providing /usr/local/cuda).

  Selection    Path                  Priority   Status
------------------------------------------------------------
* 0            /usr/local/cuda-12.2   122       auto mode
  1            /usr/local/cuda-11.3   113       manual mode
  2            /usr/local/cuda-12.2   122       manual mode

Press <enter> to keep the current choice[*], or type selection number: 1
update-alternatives: using /usr/local/cuda-11.3 to provide /usr/local/cuda (cuda) in manual mode


In [52]:
! gcc -v

Using built-in specs.
COLLECT_GCC=gcc
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/9/lto-wrapper
OFFLOAD_TARGET_NAMES=nvptx-none:hsa
OFFLOAD_TARGET_DEFAULT=1
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu 9.5.0-1ubuntu1~22.04' --with-bugurl=file:///usr/share/doc/gcc-9/README.Bugs --enable-languages=c,ada,c++,go,brig,d,fortran,objc,obj-c++,gm2 --prefix=/usr --with-gcc-major-version-only --program-suffix=-9 --program-prefix=x86_64-linux-gnu- --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --libdir=/usr/lib --enable-nls --enable-bootstrap --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --with-default-libstdcxx-abi=new --enable-gnu-unique-object --disable-vtable-verify --enable-plugin --enable-default-pie --with-system-zlib --with-target-system-zlib=auto --enable-objc-gc=auto --enable-multiarch --disable-werror --with-arch-32=i686 --with-abi=m64 --with-mu

Yet, when I try to build_cutlass.sh, the CUDA compiler is still identified as 12.2.140. The way to fix this is the cell above, changing the alternatives for CUDA usage

In [57]:
!sudo update-alternatives --display gcc

gcc - auto mode
  link best version is /usr/bin/gcc-9
  link currently points to /usr/bin/gcc-9
  link gcc is /usr/bin/gcc
  slave g++ is /usr/bin/g++
/usr/bin/gcc-9 - priority 90
  slave g++: /usr/bin/g++-9


In [61]:
! export CC=/path/to/gcc-9.5
! export CXX=/path/to/g++-9.5

In [ ]:
! set(CMAKE_C_COMPILER /path/to/gcc-9.5)
! set(CMAKE_CXX_COMPILER /path/to/g++-9.5)

In [65]:
%cd /usr/bin

/usr/bin


This still gave me errors for a bit, because you need to use a GCC compatible with the different CUDA version. But even when I installed a new GCC version, it seemed to use it but didn't? Weird.

In [49]:
! source environment.sh
! bash build_cutlass.sh   # this one needs GPU
! python setup.py install

-- CMake Version: 3.30.1
-- The CXX compiler identification is GNU 9.5.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The CUDA compiler identification is NVIDIA 11.3.58
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- CUDART: /usr/local/cuda/lib64/libcudart.so
-- CUDA Driver: /usr/local/cuda/lib64/stubs/libcuda.so
-- NVRTC: /usr/local/cuda/lib64/libnvrtc.so
-- Default Install Location: install
-- CUDA Compilation Architectures: 80
-- Enable caching of reference results in conv unit tests
-- Enable rigorous conv problem sizes in conv unit tests
-- Using NVCC flags: -DCUTLASS_TEST_LEVEL=0;-DCUTLASS_TEST_ENABLE_CACHED_RESULTS=1;-DCUTLASS_CONV_UNI

In [58]:
! python -c "import torch; print(torch.__version__)"

1.12.1+cu113


In [59]:
import smoothquant

In [60]:
from smoothquant.opt import Int8OPTForCausalLM
model = Int8OPTForCausalLM.from_pretrained("mit-han-lab/opt-30b-smoothquant")

ModuleNotFoundError: No module named 'smoothquant.opt'